In [9]:
# Import libraries
import numpy as np
import sqlite3
import random
import ast
import pandas as pd

In [10]:
dataframes = [] #Here will import that raw data from the article

min_num_vertices = 4
max_num_vertices = 4 #Set to 3 when generating everything from 3-6

#Import Polygon Data. # It verifies we have the same number of vertices across the database.
with sqlite3.connect("../data/dim_2_plucker.db") as db: #...ensure this is the correct path to the datafile
    for i in range(min_num_vertices, max_num_vertices+1): #Iterate through n=3 to n=6, where n is the number of vertices
      #c = db.cursor()
      query = "SELECT * FROM dim_2_plucker WHERE num_vertices = {}".format(i)
      dataframes.append(pd.read_sql_query(query, db))   #...read database into a pandas dataframe
      headings = dataframes[(i - min_num_vertices)].columns.values     #...save the headings of each column in the table
      data = dataframes[(i - min_num_vertices)].values

In [11]:
dataframes[0]

,id,vertices,plucker,plucker_len,num_vertices,num_points,num_interior_points,volume,dual_volume,gorenstein_idx,codimension,embedding
0,469,"[[1,0],[-1,0],[0,1],[1,-1]]","[-1,-1,1,0,1,1]",6,4,5,1,4,8,1,6,"[1,1,1,1,1,1,1,1,1]"
1,470,"[[1,0],[-1,0],[0,1],[1,-1]]","[0,-1,-1,-1,-1,1]",6,4,5,1,4,8,1,6,"[1,1,1,1,1,1,1,1,1]"
2,471,"[[1,0],[-1,0],[0,1],[1,-1]]","[1,0,-1,1,1,1]",6,4,5,1,4,8,1,6,"[1,1,1,1,1,1,1,1,1]"
3,1009,"[[1,0],[-1,0],[2,5],[-5,-8]]","[-8,-8,-9,0,5,5]",6,4,18,10,26,13/10,20,10,"[1,2,2,2,3,3,3,3,4,4,4,5,5]"
4,1010,"[[1,0],[-1,0],[2,5],[-5,-8]]","[-8,-5,0,9,8,5]",6,4,18,10,26,13/10,20,10,"[1,2,2,2,3,3,3,3,4,4,4,5,5]"
...,...,...,...,...,...,...,...,...,...,...,...,...
21118,80979,"[[1,0],[116,125],[-12,-13],[-1,-1]]","[-1,13,125,1,9,8]",6,4,89,61,148,16/5,25,6,"[1,1,1,1,1,9,13,17,25]"
21119,80980,"[[1,0],[116,125],[-12,-13],[-1,-1]]","[1,-1,-9,-13,-125,8]",6,4,89,61,148,16/5,25,6,"[1,1,1,1,1,9,13,17,25]"
21120,80981,"[[1,0],[139,180],[3,4],[-27,-35]]","[-16,-180,5,4,3,35]",6,4,130,106,234,1,30,8,"[1,1,2,3,5,9,9,13,16,23,30]"
21121,80982,"[[1,0],[139,180],[3,4],[-27,-35]]","[5,16,180,3,35,4]",6,4,130,106,234,1,30,8,"[1,1,2,3,5,9,9,13,16,23,30]"


In [12]:
max_index = max_num_vertices - min_num_vertices + 1
print(max_index)

1


In [13]:
for j in range(0, max_index):
  #selects the features that we are looking into, and eliminates duplicates
  new_df = dataframes[j][['vertices', 'num_vertices', 'volume']]
  new_df['vertices_1'] = new_df['vertices'].apply(lambda x: ast.literal_eval(x))
  #new_df = new_df.drop_duplicates(subset='vertices_1')
  new_df['coordinates_1'] =   new_df['vertices_1'].apply(lambda x: [item for sublist in x for item in sublist])

  V_dfs = []

  for k in range(2,3+1): #number of random orientations of the vertices
    vert = 'vertices_{}'.format(k)
    coord = 'coordinates_{}'.format(k)
    new_df[vert]  =   new_df['vertices_1'].apply(lambda x: random.sample(x, len(x)))
    new_df[coord] =   new_df[vert].apply(lambda x: [item for sublist in x for item in sublist])

    V_df = new_df[[vert, coord, "volume"]]
    V_df = V_df.rename(columns={vert: 'vertices', coord: 'coordinates'})

    V_dfs.append(V_df)

  Final_df = pd.concat(V_dfs, axis=0)

/var/folders/15/vdzhjh015bg5yt68s5n8y4bc0000gn/T/ipykernel_60874/2026108050.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['vertices_1'] = new_df['vertices'].apply(lambda x: ast.literal_eval(x))
/var/folders/15/vdzhjh015bg5yt68s5n8y4bc0000gn/T/ipykernel_60874/2026108050.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['coordinates_1'] =   new_df['vertices_1'].apply(lambda x: [item for sublist in x for item in sublist])
/var/folders/15/vdzhjh015bg5yt68s5n8y4bc0000gn/T/ipykernel_60874/20

In [14]:
Final_df.tail(2)

,vertices,coordinates,volume
21121,"[[-27, -35], [1, 0], [3, 4], [139, 180]]","[-27, -35, 1, 0, 3, 4, 139, 180]",234
21122,"[[3, 4], [139, 180], [1, 0], [-27, -35]]","[3, 4, 139, 180, 1, 0, -27, -35]",234


In [15]:
# Next, we creates a gigantic database with coordinates and vertices for all the transformed databases B_k*D
#It goes until

k_max = 100

for i in range(0, k_max+1):
    # Define the name of the new columns
    new_column_name = f'verticesB_{i}'
    new_coordinate_name = f'coordinatesB_{i}'

    # Define the matrix B
    B = np.array([[1, i], [0, 1]])

    # Define a lambda function to multiply each element of the nested list by B
    multiply_by_B = lambda lst: [np.dot(B, v) for v in lst]

    # Create a new column
    Final_df[new_column_name]     =   Final_df['vertices'].apply(multiply_by_B)
    Final_df[new_coordinate_name] =   Final_df[new_column_name].apply(lambda x: [item for sublist in x for item in sublist])

Final_df.head()

/var/folders/15/vdzhjh015bg5yt68s5n8y4bc0000gn/T/ipykernel_60874/1680532525.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Final_df[new_column_name]     =   Final_df['vertices'].apply(multiply_by_B)
/var/folders/15/vdzhjh015bg5yt68s5n8y4bc0000gn/T/ipykernel_60874/1680532525.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Final_df[new_coordinate_name] =   Final_df[new_column_name].apply(lambda x: [item for sublist in x for item in sublist])
/var/folders/15/vdzhjh015bg5yt68s5n8y4bc0000gn/T/ipykernel_60874/1680532525.py:

,vertices,coordinates,volume,verticesB_0,coordinatesB_0,verticesB_1,coordinatesB_1,verticesB_2,coordinatesB_2,verticesB_3,...,verticesB_96,coordinatesB_96,verticesB_97,coordinatesB_97,verticesB_98,coordinatesB_98,verticesB_99,coordinatesB_99,verticesB_100,coordinatesB_100
0,"[[0, 1], [1, -1], [1, 0], [-1, 0]]","[0, 1, 1, -1, 1, 0, -1, 0]",4,"[[0, 1], [1, -1], [1, 0], [-1, 0]]","[0, 1, 1, -1, 1, 0, -1, 0]","[[1, 1], [0, -1], [1, 0], [-1, 0]]","[1, 1, 0, -1, 1, 0, -1, 0]","[[2, 1], [-1, -1], [1, 0], [-1, 0]]","[2, 1, -1, -1, 1, 0, -1, 0]","[[3, 1], [-2, -1], [1, 0], [-1, 0]]",...,"[[96, 1], [-95, -1], [1, 0], [-1, 0]]","[96, 1, -95, -1, 1, 0, -1, 0]","[[97, 1], [-96, -1], [1, 0], [-1, 0]]","[97, 1, -96, -1, 1, 0, -1, 0]","[[98, 1], [-97, -1], [1, 0], [-1, 0]]","[98, 1, -97, -1, 1, 0, -1, 0]","[[99, 1], [-98, -1], [1, 0], [-1, 0]]","[99, 1, -98, -1, 1, 0, -1, 0]","[[100, 1], [-99, -1], [1, 0], [-1, 0]]","[100, 1, -99, -1, 1, 0, -1, 0]"
1,"[[0, 1], [1, -1], [1, 0], [-1, 0]]","[0, 1, 1, -1, 1, 0, -1, 0]",4,"[[0, 1], [1, -1], [1, 0], [-1, 0]]","[0, 1, 1, -1, 1, 0, -1, 0]","[[1, 1], [0, -1], [1, 0], [-1, 0]]","[1, 1, 0, -1, 1, 0, -1, 0]","[[2, 1], [-1, -1], [1, 0], [-1, 0]]","[2, 1, -1, -1, 1, 0, -1, 0]","[[3, 1], [-2, -1], [1, 0], [-1, 0]]",...,"[[96, 1], [-95, -1], [1, 0], [-1, 0]]","[96, 1, -95, -1, 1, 0, -1, 0]","[[97, 1], [-96, -1], [1, 0], [-1, 0]]","[97, 1, -96, -1, 1, 0, -1, 0]","[[98, 1], [-97, -1], [1, 0], [-1, 0]]","[98, 1, -97, -1, 1, 0, -1, 0]","[[99, 1], [-98, -1], [1, 0], [-1, 0]]","[99, 1, -98, -1, 1, 0, -1, 0]","[[100, 1], [-99, -1], [1, 0], [-1, 0]]","[100, 1, -99, -1, 1, 0, -1, 0]"
2,"[[-1, 0], [1, 0], [1, -1], [0, 1]]","[-1, 0, 1, 0, 1, -1, 0, 1]",4,"[[-1, 0], [1, 0], [1, -1], [0, 1]]","[-1, 0, 1, 0, 1, -1, 0, 1]","[[-1, 0], [1, 0], [0, -1], [1, 1]]","[-1, 0, 1, 0, 0, -1, 1, 1]","[[-1, 0], [1, 0], [-1, -1], [2, 1]]","[-1, 0, 1, 0, -1, -1, 2, 1]","[[-1, 0], [1, 0], [-2, -1], [3, 1]]",...,"[[-1, 0], [1, 0], [-95, -1], [96, 1]]","[-1, 0, 1, 0, -95, -1, 96, 1]","[[-1, 0], [1, 0], [-96, -1], [97, 1]]","[-1, 0, 1, 0, -96, -1, 97, 1]","[[-1, 0], [1, 0], [-97, -1], [98, 1]]","[-1, 0, 1, 0, -97, -1, 98, 1]","[[-1, 0], [1, 0], [-98, -1], [99, 1]]","[-1, 0, 1, 0, -98, -1, 99, 1]","[[-1, 0], [1, 0], [-99, -1], [100, 1]]","[-1, 0, 1, 0, -99, -1, 100, 1]"
3,"[[1, 0], [2, 5], [-1, 0], [-5, -8]]","[1, 0, 2, 5, -1, 0, -5, -8]",26,"[[1, 0], [2, 5], [-1, 0], [-5, -8]]","[1, 0, 2, 5, -1, 0, -5, -8]","[[1, 0], [7, 5], [-1, 0], [-13, -8]]","[1, 0, 7, 5, -1, 0, -13, -8]","[[1, 0], [12, 5], [-1, 0], [-21, -8]]","[1, 0, 12, 5, -1, 0, -21, -8]","[[1, 0], [17, 5], [-1, 0], [-29, -8]]",...,"[[1, 0], [482, 5], [-1, 0], [-773, -8]]","[1, 0, 482, 5, -1, 0, -773, -8]","[[1, 0], [487, 5], [-1, 0], [-781, -8]]","[1, 0, 487, 5, -1, 0, -781, -8]","[[1, 0], [492, 5], [-1, 0], [-789, -8]]","[1, 0, 492, 5, -1, 0, -789, -8]","[[1, 0], [497, 5], [-1, 0], [-797, -8]]","[1, 0, 497, 5, -1, 0, -797, -8]","[[1, 0], [502, 5], [-1, 0], [-805, -8]]","[1, 0, 502, 5, -1, 0, -805, -8]"
4,"[[-5, -8], [-1, 0], [1, 0], [2, 5]]","[-5, -8, -1, 0, 1, 0, 2, 5]",26,"[[-5, -8], [-1, 0], [1, 0], [2, 5]]","[-5, -8, -1, 0, 1, 0, 2, 5]","[[-13, -8], [-1, 0], [1, 0], [7, 5]]","[-13, -8, -1, 0, 1, 0, 7, 5]","[[-21, -8], [-1, 0], [1, 0], [12, 5]]","[-21, -8, -1, 0, 1, 0, 12, 5]","[[-29, -8], [-1, 0], [1, 0], [17, 5]]",...,"[[-773, -8], [-1, 0], [1, 0], [482, 5]]","[-773, -8, -1, 0, 1, 0, 482, 5]","[[-781, -8], [-1, 0], [1, 0], [487, 5]]","[-781, -8, -1, 0, 1, 0, 487, 5]","[[-789, -8], [-1, 0], [1, 0], [492, 5]]","[-789, -8, -1, 0, 1, 0, 492, 5]","[[-797, -8], [-1, 0], [1, 0], [497, 5]]","[-797, -8, -1, 0, 1, 0, 497, 5]","[[-805, -8], [-1, 0], [1, 0], [502, 5]]","[-805, -8, -1, 0, 1, 0, 502, 5]"


In [17]:
database_name = '../data/{0}Vertices_k{1}.db'.format(1 + 3, k_max)
conn = sqlite3.connect(database_name)
data_toSQL = Final_df.applymap(str)
data_toSQL.to_sql('Data', conn, if_exists='replace')
conn.close()

#Checking the database has the correct Polygon Data.
conn = sqlite3.connect(database_name)
# Read a table from the database into a Pandas DataFrame
check_df = pd.read_sql('SELECT * FROM Data', conn)
# Close the database connection
conn.close()
check_df.head()

/var/folders/15/vdzhjh015bg5yt68s5n8y4bc0000gn/T/ipykernel_60874/23066005.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_toSQL = Final_df.applymap(str)


,index,vertices,coordinates,volume,verticesB_0,coordinatesB_0,verticesB_1,coordinatesB_1,verticesB_2,coordinatesB_2,...,verticesB_96,coordinatesB_96,verticesB_97,coordinatesB_97,verticesB_98,coordinatesB_98,verticesB_99,coordinatesB_99,verticesB_100,coordinatesB_100
0,0,"[[0, 1], [1, -1], [1, 0], [-1, 0]]","[0, 1, 1, -1, 1, 0, -1, 0]",4,"[array([0, 1]), array([ 1, -1]), array([1, 0])...","[0, 1, 1, -1, 1, 0, -1, 0]","[array([1, 1]), array([ 0, -1]), array([1, 0])...","[1, 1, 0, -1, 1, 0, -1, 0]","[array([2, 1]), array([-1, -1]), array([1, 0])...","[2, 1, -1, -1, 1, 0, -1, 0]",...,"[array([96, 1]), array([-95, -1]), array([1,...","[96, 1, -95, -1, 1, 0, -1, 0]","[array([97, 1]), array([-96, -1]), array([1,...","[97, 1, -96, -1, 1, 0, -1, 0]","[array([98, 1]), array([-97, -1]), array([1,...","[98, 1, -97, -1, 1, 0, -1, 0]","[array([99, 1]), array([-98, -1]), array([1,...","[99, 1, -98, -1, 1, 0, -1, 0]","[array([100, 1]), array([-99, -1]), array([...","[100, 1, -99, -1, 1, 0, -1, 0]"
1,1,"[[0, 1], [1, -1], [1, 0], [-1, 0]]","[0, 1, 1, -1, 1, 0, -1, 0]",4,"[array([0, 1]), array([ 1, -1]), array([1, 0])...","[0, 1, 1, -1, 1, 0, -1, 0]","[array([1, 1]), array([ 0, -1]), array([1, 0])...","[1, 1, 0, -1, 1, 0, -1, 0]","[array([2, 1]), array([-1, -1]), array([1, 0])...","[2, 1, -1, -1, 1, 0, -1, 0]",...,"[array([96, 1]), array([-95, -1]), array([1,...","[96, 1, -95, -1, 1, 0, -1, 0]","[array([97, 1]), array([-96, -1]), array([1,...","[97, 1, -96, -1, 1, 0, -1, 0]","[array([98, 1]), array([-97, -1]), array([1,...","[98, 1, -97, -1, 1, 0, -1, 0]","[array([99, 1]), array([-98, -1]), array([1,...","[99, 1, -98, -1, 1, 0, -1, 0]","[array([100, 1]), array([-99, -1]), array([...","[100, 1, -99, -1, 1, 0, -1, 0]"
2,2,"[[-1, 0], [1, 0], [1, -1], [0, 1]]","[-1, 0, 1, 0, 1, -1, 0, 1]",4,"[array([-1, 0]), array([1, 0]), array([ 1, -1...","[-1, 0, 1, 0, 1, -1, 0, 1]","[array([-1, 0]), array([1, 0]), array([ 0, -1...","[-1, 0, 1, 0, 0, -1, 1, 1]","[array([-1, 0]), array([1, 0]), array([-1, -1...","[-1, 0, 1, 0, -1, -1, 2, 1]",...,"[array([-1, 0]), array([1, 0]), array([-95, ...","[-1, 0, 1, 0, -95, -1, 96, 1]","[array([-1, 0]), array([1, 0]), array([-96, ...","[-1, 0, 1, 0, -96, -1, 97, 1]","[array([-1, 0]), array([1, 0]), array([-97, ...","[-1, 0, 1, 0, -97, -1, 98, 1]","[array([-1, 0]), array([1, 0]), array([-98, ...","[-1, 0, 1, 0, -98, -1, 99, 1]","[array([-1, 0]), array([1, 0]), array([-99, ...","[-1, 0, 1, 0, -99, -1, 100, 1]"
3,3,"[[1, 0], [2, 5], [-1, 0], [-5, -8]]","[1, 0, 2, 5, -1, 0, -5, -8]",26,"[array([1, 0]), array([2, 5]), array([-1, 0])...","[1, 0, 2, 5, -1, 0, -5, -8]","[array([1, 0]), array([7, 5]), array([-1, 0])...","[1, 0, 7, 5, -1, 0, -13, -8]","[array([1, 0]), array([12, 5]), array([-1, 0...","[1, 0, 12, 5, -1, 0, -21, -8]",...,"[array([1, 0]), array([482, 5]), array([-1, ...","[1, 0, 482, 5, -1, 0, -773, -8]","[array([1, 0]), array([487, 5]), array([-1, ...","[1, 0, 487, 5, -1, 0, -781, -8]","[array([1, 0]), array([492, 5]), array([-1, ...","[1, 0, 492, 5, -1, 0, -789, -8]","[array([1, 0]), array([497, 5]), array([-1, ...","[1, 0, 497, 5, -1, 0, -797, -8]","[array([1, 0]), array([502, 5]), array([-1, ...","[1, 0, 502, 5, -1, 0, -805, -8]"
4,4,"[[-5, -8], [-1, 0], [1, 0], [2, 5]]","[-5, -8, -1, 0, 1, 0, 2, 5]",26,"[array([-5, -8]), array([-1, 0]), array([1, 0...","[-5, -8, -1, 0, 1, 0, 2, 5]","[array([-13, -8]), array([-1, 0]), array([1,...","[-13, -8, -1, 0, 1, 0, 7, 5]","[array([-21, -8]), array([-1, 0]), array([1,...","[-21, -8, -1, 0, 1, 0, 12, 5]",...,"[array([-773, -8]), array([-1, 0]), array([...","[-773, -8, -1, 0, 1, 0, 482, 5]","[array([-781, -8]), array([-1, 0]), array([...","[-781, -8, -1, 0, 1, 0, 487, 5]","[array([-789, -8]), array([-1, 0]), array([...","[-789, -8, -1, 0, 1, 0, 492, 5]","[array([-797, -8]), array([-1, 0]), array([...","[-797, -8, -1, 0, 1, 0, 497, 5]","[array([-805, -8]), array([-1, 0]), array([...","[-805, -8, -1, 0, 1, 0, 502, 5]"
